In [61]:
import pandas as pd
import csv
import matplotlib.pyplot as plt
import math
import numpy as np

In [19]:
def uniqueWords(data):
    allWords = []
    for i in range(len(data)):
        for j in range(2, len(data[i])): # from 2 in order to exclude label and file name
            allWords.append(data[i][j])
    
    vc = pd.Series(allWords).value_counts()
    uniques=vc.index.values.tolist()        
    return allWords, uniques, vc

In [31]:
def labelFrequencies(data):
    values = []
    for i in range(len(data)):
        values.append(data[i][0])
    serVal = pd.DataFrame(values)
    return serVal

In [34]:
def getLengths(data):
    lens = []
    for i in range(len(data)):
        lens.append(len(data[i])-2)
    lens = pd.DataFrame(lens)
    lensFreq = lens[0].value_counts()
    lensFreq=pd.DataFrame(lensFreq)
    return lens, lensFreq

In [50]:
def getStd(len_sorted, med):
    res=0
    for i in range(len(len_sorted)):
        x = len_sorted.iloc[i,0]
        res = res + (x-med)*(x-med)
    res2 = res / len(len_sorted)
    res_final = math.sqrt(res2)
    print("stdev= ", res_final)
    return res_final

In [27]:
# filename = "myData.csv"
# with open(filename, newline='', encoding='utf-8') as f:
#     reader = csv.reader(f)
#     data = list(reader)
# data = dropEmpty(data)

In [3]:
def optLength(data, filename):
    
    #find the words, the unique words and their frequencies       
    allWords, uniques, tokenFreq = uniqueWords(data)
    tokenFreq = pd.DataFrame(tokenFreq)
    l=tokenFreq.index.values.tolist()
    tokenFreq.insert(0,'token', l)
    tokenFreq.rename(columns = {0: 'times'}, inplace = True)
    tokenFreq = tokenFreq.reset_index(drop=True)
    #avgToken = sum(tokenFreq['times']) / len(tokenFreq['times'])
    #maxToken = max(tokenFreq['times'])
    #minToken = min(tokenFreq['times'])
    #write uniques to file
    textfile = open(filename + "_uniques.txt", "w", encoding='utf-8')
    for element in uniques:
        textfile.write(element + "\n")
    textfile.close()
    
    #check balance
    labels = labelFrequencies(data)
    labelFreq = labels[0].value_counts()
    print("labels balance:\n",labelFreq)
    
    # find the length of each class file
    lens, lensFreq = getLengths(data)
    l=lensFreq.index.values.tolist()
    
    lensFreq.insert(0,'lengths', l)
    lensFreq.rename(columns = {0: 'times'}, inplace = True)
    lensFreq = lensFreq.reset_index(drop=True)
    
    avgLen = sum(lens.iloc[:,0]) / len(lens.iloc[:,0])# mesos oros twn sequence lengths
    print("avg of lengths = ",avgLen)
    print("max of lengths = ", max(lens.iloc[:,0]))
    print("min of lengths = ", min(lens.iloc[:,0]))
    
    len_sorted = lens.sort_values(by=[0])
    len_sorted = len_sorted.reset_index(drop=True)
    
    med = len_sorted.median()[0]
    print("median= ", med)
    
    plt.plot(len_sorted[0])
    plt.show()
    
    stdev = len_sorted[0].std()
    print("standard deviation = ", stdev)
    
    #manual computation of st. d.
    #stdev2 = getStd(len_sorted, med)
    
    split_point = int(med + 2*stdev)
    print("median + 2*st.d= ", split_point)
    max_len_std = len_sorted.iloc[split_point-1,:][0]
    print("max length selected, based on median + 2*st.d: ", max_len_std)
    
    #boxplot = len_sorted.boxplot(column=[0])
    plt.boxplot(len_sorted[0])
    plt.show()
    
    # finding the 1st and 3rd quartiles
    q1 = np.quantile(len_sorted[0], 0.25)
    q3 = np.quantile(len_sorted[0], 0.75)
    # finding the iqr region
    iqr = q3-q1
    # finding upper and lower whiskers
    upper_bound = q3+(1.5*iqr)
    lower_bound = q1-(1.5*iqr)
    print(iqr, upper_bound, lower_bound)
    outliers = len_sorted[0][(len_sorted[0] <= lower_bound) | (len_sorted[0] >= upper_bound)]
    split_point = len(len_sorted) - len(outliers)
    print("IQR-based splitting point: ", split_point)
    max_len_iqr = len_sorted.iloc[split_point-1,:][0]
    print("max length selected, based on IQR: ", max_len_iqr)
    
    #reshape dataset
    max_len = max_len_iqr # or max_len_std
    data2 = []
    for i in range(len(data)):
        length = len(data[i]) - 2
        if length <= max_len :
            data2.append(data[i])
    
    #save reduced dataset
    print("Save dataset to file...")
    with open(filename + "_reduced.csv","w", encoding='utf-8') as f:
        wr = csv.writer(f)
        wr.writerows(data2)
    
    return data2